In [ ]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install accelerate
!pip install huggingface_hub


INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 26.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.9 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset
from datasets import Dataset

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments , pipeline

## Preparing Arabic Data

In [ ]:


ds_1 = load_dataset("M-A-D/Mixed-Arabic-Datasets-Repo", "Ara--Goud--Goud-sum")
ds_2 = load_dataset("M-A-D/Mixed-Arabic-Datasets-Repo", "Ara--J-Mourad--MNAD.v1")
ds_3 = load_dataset("M-A-D/Mixed-Arabic-Datasets-Repo", "Ara--saudinewsnet")


README.md:   0%|          | 0.00/16.0k [00:00<?, ?B/s]

(…)-00000-of-00001-f4548f25552a55c9.parquet:   0%|          | 0.00/148M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/139288 [00:00<?, ? examples/s]

(…)-00000-of-00003-c18492ac8270b0e4.parquet:   0%|          | 0.00/205M [00:00<?, ?B/s]

(…)-00001-of-00003-0f9b962e03a0cf63.parquet:   0%|          | 0.00/190M [00:00<?, ?B/s]

(…)-00002-of-00003-438ca413ea7b6ff8.parquet:   0%|          | 0.00/133M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/418563 [00:00<?, ? examples/s]

(…)-00000-of-00001-4de2ec74c49f39fb.parquet:   0%|          | 0.00/49.1M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/31030 [00:00<?, ? examples/s]

In [ ]:
print(ds_1)
print(ds_2)
print(ds_3)

DatasetDict({
    train: Dataset({
        features: ['article', 'headline', 'categories'],
        num_rows: 139288
    })
})
DatasetDict({
    train: Dataset({
        features: ['Title', 'Body', 'Category'],
        num_rows: 418563
    })
})
DatasetDict({
    train: Dataset({
        features: ['source', 'url', 'date_extracted', 'title', 'author', 'content'],
        num_rows: 31030
    })
})


In [ ]:
# ds_1
ds_1 = ds_1.rename_column("article", "Body")
ds_1 = ds_1.rename_column("headline", "Title")

ds_1 = ds_1.remove_columns(['categories'])

In [ ]:
ds_2 = ds_2.remove_columns(['Category'])

In [ ]:
ds_3 = ds_3.remove_columns(['source', 'url', 'date_extracted', 'author'])
# rename columns
ds_3 = ds_3.rename_column("content", "Body")
ds_3 = ds_3.rename_column("title", "Title")

In [ ]:
from datasets import concatenate_datasets, DatasetDict

ds = DatasetDict()
ds['train'] = concatenate_datasets([ds_1['train'], ds_2['train'] , ds_3['train']])

ds = ds.shuffle()


In [ ]:
ar_data = ds

In [ ]:
ar_data

DatasetDict({
    train: Dataset({
        features: ['Body', 'Title'],
        num_rows: 588881
    })
})

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) Y
Token is valid (permission: fineGrained).
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub.
Run the following command in your termin

In [ ]:
ar_data.push_to_hub("YoussefAnwar/Arabic-news")

Uploading the dataset shards:   0%|          | 0/3 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/197 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/197 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/197 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/YoussefAnwar/Arabic-news/commit/611a631f020a8d5e047a8cdb323a057ee855e422', commit_message='Upload dataset', commit_description='', oid='611a631f020a8d5e047a8cdb323a057ee855e422', pr_url=None, pr_revision=None, pr_num=None)

In [ ]:
# https://huggingface.co/datasets/vgoldberg/longform_article_summarization

# https://huggingface.co/datasets/Stefan171/TheGuardian-Articles

# https://huggingface.co/datasets/rjac/all-the-news-2-1-Component-one


## Preparing English Data

In [ ]:
eng_data = load_dataset('rjac/all-the-news-2-1-Component-one')

README.md:   0%|          | 0.00/814 [00:00<?, ?B/s]

Resolving data files:   0%|          | 0/36 [00:00<?, ?it/s]

dataset_infos.json:   0%|          | 0.00/1.35k [00:00<?, ?B/s]

(…)-00000-of-00036-44ab9c81d9134716.parquet:   0%|          | 0.00/193M [00:00<?, ?B/s]

(…)-00001-of-00036-f36e9ee23e3bf9b5.parquet:   0%|          | 0.00/192M [00:00<?, ?B/s]

...

(…)-00033-of-00036-5df217a26c2a9435.parquet:   0%|          | 0.00/127M [00:00<?, ?B/s]

(…)-00034-of-00036-5afd8a909cd67764.parquet:   0%|          | 0.00/123M [00:00<?, ?B/s]

(…)-00035-of-00036-a2f21c2111e3bf04.parquet:   0%|          | 0.00/125M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2688878 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/18 [00:00<?, ?it/s]

In [ ]:
deleted_columns = ['idx', 'article_idx', 'date', 'year', 'month', 'day', 'author', 'url', 'section', 'publication']

eng_data = eng_data.remove_columns(deleted_columns)

eng_data = eng_data.rename_column("title", "Title")
eng_data = eng_data.rename_column("article", "Body")

In [ ]:
eng_data

DatasetDict({
    train: Dataset({
        features: ['Title', 'Body'],
        num_rows: 2688878
    })
})

In [ ]:
eng_data.push_to_hub("YoussefAnwar/English-news")

Uploading the dataset shards:   0%|          | 0/17 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

...

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

Creating parquet from Arrow format:   0%|          | 0/159 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/YoussefAnwar/English-news/commit/185a9cc218fba0f0cf87d2b91a05663cae66a5e9', commit_message='Upload dataset', commit_description='', oid='185a9cc218fba0f0cf87d2b91a05663cae66a5e9', pr_url=None, pr_revision=None, pr_num=None)